Monte carlo estimation for different market states (Bear, Bull and Neutral)

In [10]:
# Help in market regime classification - based on the state returns

In [ ]:
import numpy as np
states = ['bull', 'bear', 'neutral']
num_states = len(states)
state_returns = {"bull": 0.02, "bear": -0.015, "neutral": 0.005} 

In [11]:
num_episodes = 1000000
num_days = 30

In [12]:
state_values = {state: [] for state in states}

In [13]:
for _ in range(num_episodes):
    episode = []
    total_return = 0
    visited_states = set()
    
    current_state = np.random.choice(states)
    
    for _ in range(num_days):
        if current_state not in visited_states:
            episode.append((current_state, total_return))
            visited_states.add(current_state)
        
        total_return += state_returns[current_state]
        
        # Market state transition withing num_days -> 0.3
        if np.random.rand() < 0.3:
            current_state = np.random.choice(states)
    
    for state, ret in episode:
        state_values[state].append(total_return)
            

In [16]:
V = {state: np.mean(state_values[state]) if state_values[state] else 0 for state in states}

In [17]:
print("Monte Carlo Estimated State Values:")
for state in states:
    print(f"  {state.capitalize()} Market: {V[state]:.4f}")

Monte Carlo Estimated State Values:
  Bull Market: 0.1081
  Bear Market: 0.0912
  Neutral Market: 0.1009


GPU Implementation

In [ ]:
import cupy as cp

# Map states to integers
state_map = {"bull": 0, "bear": 1, "neutral": 2}
num_states = len(state_map)

state_returns_int = {0: 0.02, 1: -0.015, 2: 0.005}
returns_gpu = cp.array([state_returns_int[i] for i in range(num_states)], dtype=cp.float32)

num_episodes = 1_000_000_00
num_days = 30
transition_prob = 0.2

state_values_gpu = cp.zeros((num_states, num_episodes), dtype=cp.float32)

def monte_carlo_gpu():
    current_states = cp.random.choice(num_states, size=num_episodes)
    total_returns = cp.zeros(num_episodes, dtype=cp.float32)
    
    for _ in range(num_days):
        total_returns += returns_gpu[current_states]
        mask = cp.random.rand(num_episodes) < transition_prob
        new_states = cp.random.choice(num_states, size=num_episodes)
        current_states = cp.where(mask, new_states, current_states)
    
    for i in range(num_states):
        mask = current_states == i
        state_values_gpu[i, mask] = total_returns[mask]

monte_carlo_gpu()

V_gpu = cp.mean(state_values_gpu, axis=1).get()

market_names = {0: "bull", 1: "bear", 2: "neutral"}
for i, value in enumerate(V_gpu):
    print(f"{market_names[i].capitalize()} Market: {value:.4f}")


Bull Market: 0.0555
Bear Market: 0.0089
Neutral Market: 0.0356
